In [1]:
import os
import ee
import pandas as pd

In [22]:
data = pd.read_csv("csv/GHG_Data_Sample.csv")

In [23]:
data['Date'] = pd.to_datetime(data['Date'], format = '%m/%d/%y')
data.head()

,Date,Site,ID,Lat,Long,CO2PixelDay
0,2015-07-15,Goodrich,R1,40.366021,-120.949558,80.613691
1,2015-07-15,Goodrich,R10,40.365042,-120.949802,82.194613
2,2015-07-15,Goodrich,R11,40.365238,-120.950044,44.048585
3,2015-07-15,Goodrich,R12,40.365436,-120.950289,94.443643
4,2015-07-15,Goodrich,R2,40.365824,-120.949316,157.806186


In [2]:
#ee.Authenticate()
ee.Initialize()

In [3]:
def maskClouds(image):
    quality = image.select('pixel_qa')
    cloud = quality.bitwiseAnd(1 << 5).eq(0)    # mask out cloud shadow
    clear = quality.bitwiseAnd(1 << 4).eq(0)     # mask out cloud
    return image.updateMask(cloud).updateMask(clear)

In [4]:
def calculate_time_difference(image):
    time_difference = ee.Number(image.date().difference(target_date, 'day')).abs()
    return image.set('time_difference', time_difference)

In [5]:
def getBandValues(landsat_collection, point, target_date, bufferDays = 30, landsatNo = 8):
    # filter landsat images by location
    spatial_filtered = landsat_collection.filterBounds(point)
    # filter the streamlined images by dates +/- a certain number of days
    temporal_filtered = spatial_filtered.filterDate(ee.Date(target_date).advance(-bufferDays, 'day'), ee.Date(target_date).advance(bufferDays, 'day'))
    # apply cloud mask and sort images in the collection
    cloud_free_images = temporal_filtered.map(maskClouds)
    # Map the ImageCollection over time difference and sort by that property
    sorted_collection = cloud_free_images.map(calculate_time_difference).sort('time_difference')
    image_list = sorted_collection.toList(sorted_collection.size())
    noImages = image_list.size().getInfo()
    nImage, band_values = 0, {'B2': None}
    
    # repeatedly check for cloud free pixels (non-null value) in landsat 8, or checks in landsat 7
    while band_values['B2'] == None and nImage < noImages:
        nearest_image = ee.Image(image_list.get(nImage))
        nImage += 1
        if landsatNo == 7:
            bands = nearest_image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
        else:
            bands = nearest_image.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
        properties = nearest_image.getInfo()['properties']
        band_values = bands.reduceRegion(ee.Reducer.mean(), point, 30).getInfo()
    
    return [list(band_values.values()), properties['time_difference'], properties['SENSING_TIME']]

In [6]:
landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
landsat7_collection = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
gridmet = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")

In [9]:
# define arrays to store band values and landsat information
Blue, Green, Red, NIR, SWIR_1, SWIR_2 = [], [], [], [], [], []
acq_date, acq_time, driver, time_diff = [], [], [], []

# populate bands by applying above functions for each pixel in dataframe
for id in range(data.shape[0]):
    x, y = data.loc[id, ['Long', 'Lat']]
    point = ee.Geometry.Point(x, y)
    target_date = data.loc[id, 'Date'].strftime('%Y-%m-%d')
    # 30 day radius used to search for cloud-free images
    vxn = 8
    band_values, t_diff, l_epoch = getBandValues(landsat8_collection, point, target_date, 30)
    if not band_values[0]:
        vxn = 7
        band_values, t_diff, l_epoch = getBandValues(landsat7_collection, point, target_date, 30, 7)
        # 60 day radius used to find more cloud-free images
        if not band_values[0]:
            vxn = 8
            print(id, "Searching Landsat 8 collection with 60-day search radius")
            band_values, t_diff, l_epoch = getBandValues(landsat8_collection, point, target_date, 60)
            if not band_values[0]:
                vxn = 7
                print(id, "Searching Landsat 7 collection with 60-day search radius")
                band_values, t_diff, l_epoch = getBandValues(landsat7_collection, point,
                                                                 target_date, 60, 7)
        
    Blue.append(band_values[0])
    Green.append(band_values[1])
    Red.append(band_values[2])
    NIR.append(band_values[3])
    SWIR_1.append(band_values[4])
    SWIR_2.append(band_values[5])
    driver.append(vxn)
    time_diff.append(t_diff)
    acq_date.append(l_epoch.split('T')[0])
    acq_time.append(l_epoch.split('T')[1].split('.')[0])
    
    if id%100 == 0: print(id, end=' ')


data['Blue'] = Blue
data['Green'] = Green
data['Red'] = Red
data['NIR'] = NIR
data['SWIR_1'] = SWIR_1
data['SWIR_2'] = SWIR_2
data['Acquisition_Date'] = acq_date
data['Acquisition_Time'] = acq_time
data['Driver'] = driver
data['Days_of_data_acquisition_offset'] = time_diff

data.head(10)

0 100 200 300 400 500 600 673 Searching Landsat 8 collection with 60-day search radius
674 Searching Landsat 8 collection with 60-day search radius
675 Searching Landsat 8 collection with 60-day search radius
676 Searching Landsat 8 collection with 60-day search radius
677 Searching Landsat 8 collection with 60-day search radius
678 Searching Landsat 8 collection with 60-day search radius
679 Searching Landsat 8 collection with 60-day search radius
680 Searching Landsat 8 collection with 60-day search radius
681 Searching Landsat 8 collection with 60-day search radius
682 Searching Landsat 8 collection with 60-day search radius
683 Searching Landsat 8 collection with 60-day search radius
684 Searching Landsat 8 collection with 60-day search radius
685 Searching Landsat 8 collection with 60-day search radius
686 Searching Landsat 8 collection with 60-day search radius
687 Searching Landsat 8 collection with 60-day search radius
688 Searching Landsat 8 collection with 60-day search radiu

,Date,Site,ID,Lat,Long,CO2PixelDay,Blue,Green,Red,NIR,SWIR_1,SWIR_2,Acquisition_Date,Acquisition_Time,Driver,Days_of_data_acquisition_offset
0,2015-07-15,Goodrich,R1,40.366021,-120.949558,80.613691,596.0,785.0,924.0,2058.0,2862.0,2006.0,2015-07-11,18:44:48,8,3.218888
1,2015-07-15,Goodrich,R10,40.365042,-120.949802,82.194613,568.0,763.0,865.0,2008.0,2762.0,1994.0,2015-07-11,18:44:48,8,3.218888
2,2015-07-15,Goodrich,R11,40.365238,-120.950044,44.048585,559.0,749.0,845.0,1992.0,2677.0,1901.0,2015-07-11,18:44:48,8,3.218888
3,2015-07-15,Goodrich,R12,40.365436,-120.950289,94.443643,573.0,794.0,890.0,2262.0,2871.0,1906.0,2015-07-11,18:44:48,8,3.218888
4,2015-07-15,Goodrich,R2,40.365824,-120.949316,157.806186,596.0,785.0,924.0,2058.0,2862.0,2006.0,2015-07-11,18:44:48,8,3.218888
5,2015-07-15,Goodrich,R24,40.362695,-120.952175,24.530692,327.0,517.0,578.0,1701.0,1714.0,1033.0,2015-07-11,18:44:48,8,3.218888
6,2015-07-15,Goodrich,R3,40.365629,-120.949073,82.042008,553.0,710.0,855.0,1807.0,2755.0,2036.0,2015-07-11,18:44:48,8,3.218888
7,2015-07-15,Goodrich,R4,40.365433,-120.949317,74.781880,499.0,658.0,758.0,1890.0,2599.0,1833.0,2015-07-11,18:44:48,8,3.218888
8,2015-07-15,Goodrich,R5,40.365631,-120.949561,75.322777,564.0,740.0,860.0,2006.0,2818.0,1943.0,2015-07-11,18:44:48,8,3.218888
9,2015-07-15,Goodrich,R6,40.365827,-120.949802,102.295711,577.0,778.0,916.0,2181.0,2862.0,1940.0,2015-07-11,18:44:48,8,3.218888


In [8]:
len([x for x in Blue if x])

2817

In [56]:
# write to a new csv file
data.to_csv('csv/GHG_Data_Sample_Bands.csv', index=False)

In [7]:
# GRIDMET
data = pd.read_csv("csv/GHG_Data_Sample_Bands.csv")
data.head()

,Date,Site,ID,Lat,Long,CO2PixelDay,Blue,Green,Red,NIR,SWIR_1,SWIR_2,Acquisition_Date,Acquisition_Time,Driver,Days_of_data_acquisition_offset
0,2015-07-15,Goodrich,R1,40.366021,-120.949558,80.613691,596.0,785.0,924.0,2058.0,2862.0,2006.0,2015-07-11,18:44:48,8,3.218888
1,2015-07-15,Goodrich,R10,40.365042,-120.949802,82.194613,568.0,763.0,865.0,2008.0,2762.0,1994.0,2015-07-11,18:44:48,8,3.218888
2,2015-07-15,Goodrich,R11,40.365238,-120.950044,44.048585,559.0,749.0,845.0,1992.0,2677.0,1901.0,2015-07-11,18:44:48,8,3.218888
3,2015-07-15,Goodrich,R12,40.365436,-120.950289,94.443643,573.0,794.0,890.0,2262.0,2871.0,1906.0,2015-07-11,18:44:48,8,3.218888
4,2015-07-15,Goodrich,R2,40.365824,-120.949316,157.806186,596.0,785.0,924.0,2058.0,2862.0,2006.0,2015-07-11,18:44:48,8,3.218888


In [ ]:
min_temp, max_temp = [], []

for id in range(data.shape[0]):
    x, y = data.loc[id, ['Long', 'Lat']]
    point = ee.Geometry.Point(x, y)
    target_date = data.loc[id, 'Date']
    gridmet_filtered = gridmet.filterDate(ee.Date(target_date).advance(-16, 'day'), ee.Date(target_date).advance(16, 'day'))
    bands = ee.Image(gridmet_filtered.first()).select(['tmmn', 'tmmx'])
    temperature_values = bands.reduceRegion(ee.Reducer.mean(), point, 4000).getInfo()
    tmin = temperature_values['tmmn']
    tmax = temperature_values['tmmx']
    
    min_temp.append(tmin)
    max_temp.append(tmax)
    
    if id%100 == 0: print(id, end=' ')

data['Minimum_temperature'] = min_temp
data['Maximum_temperature'] = max_temp

In [ ]:
data.to_csv('csv/GHG_Data_Sample_Bands_Temp.csv', index=False)

In [30]:
# Play around with a single landsat point and visualize
x, y = data.loc[1, ['Long', 'Lat']]
point = ee.Geometry.Point(x, y)
target_date = data.loc[1, 'Date'].strftime('%Y-%m-%d')

In [31]:
spatial_filtered = landsat8_collection.filterBounds(point)
temporal_filtered = spatial_filtered.filterDate(ee.Date(target_date).advance(-7, 'day'), ee.Date(target_date).advance(7, 'day'))

In [48]:
cloud_free_images = temporal_filtered.map(maskClouds)
sorted_collection = cloud_free_images.map(calculate_time_difference).sort('time_difference')
nearest_image = ee.Image(sorted_collection.first())

In [ ]:
import geemap

map_l8 = geemap.Map(center=[y,x], zoom=10)
image_viz_params = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

# Add the image layer to the map and display it.
map_l8.add_layer(nearest_image, image_viz_params, 'false color composite')
display(map_l8)